# Feature Engineering Basics

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

## Transformation

In [2]:
clients = pd.read_csv('clients.csv', parse_dates = ['joined'])
clients.head()

,client_id,joined,income,credit_score
0,46109,2002-04-16,172677,527
1,49545,2007-11-14,104564,770
2,41480,2013-03-11,122607,585
3,46180,2001-11-06,43851,562
4,25707,2006-10-06,211422,621


In [3]:
clients['join_month'] = clients['joined'].dt.month

clients['log_income'] = np.log(clients['income'])

clients.head()

,client_id,joined,income,credit_score,join_month,log_income
0,46109,2002-04-16,172677,527,4,12.059178
1,49545,2007-11-14,104564,770,11,11.557555
2,41480,2013-03-11,122607,585,3,11.716739
3,46180,2001-11-06,43851,562,11,10.688553
4,25707,2006-10-06,211422,621,10,12.261611


## Aggregations

In [4]:
loans = pd.read_csv('loans.csv', parse_dates = ['loan_start', 'loan_end'])

stats = loans.groupby('client_id')['loan_amount'].agg(['mean', 'max', 'min'])
stats.columns = ['mean_loan_amount', 'max_loan_amount', 'min_loan_amount']
stats.head()

,mean_loan_amount,max_loan_amount,min_loan_amount
client_id,,,
25707,7963.950000,13913,1212
26326,7270.062500,13464,1164
26695,7824.722222,14865,2389
26945,7125.933333,14593,653
29841,9813.000000,14837,2778


In [5]:
clients.merge(stats, left_on = 'client_id', right_index=True, how = 'left').head()

,client_id,joined,income,credit_score,join_month,log_income,mean_loan_amount,max_loan_amount,min_loan_amount
0,46109,2002-04-16,172677,527,4,12.059178,8951.60,14049,559
1,49545,2007-11-14,104564,770,11,11.557555,10289.30,14971,3851
2,41480,2013-03-11,122607,585,3,11.716739,7894.85,14399,811
3,46180,2001-11-06,43851,562,11,10.688553,7700.85,14081,1607
4,25707,2006-10-06,211422,621,10,12.261611,7963.95,13913,1212


## Featuretools

In [6]:
import featuretools as ft

es = ft.EntitySet(id = 'clients')

In [7]:
payments = pd.read_csv('payments.csv', parse_dates = ['payment_date'])

In [8]:
es = es.entity_from_dataframe(entity_id = 'clients', dataframe = clients, 
                              index = 'client_id', time_index = 'joined')

In [9]:
es = es.entity_from_dataframe(entity_id = 'loans', dataframe = loans, 
                              variable_types = {'repaid': ft.variable_types.Categorical},
                              index = 'loan_id', 
                              time_index = 'loan_start')

In [10]:
es = es.entity_from_dataframe(entity_id = 'payments', 
                              dataframe = payments,
                              variable_types = {'missed': ft.variable_types.Categorical},
                              make_index = True,
                              index = 'payment_id',
                              time_index = 'payment_date')


In [11]:
es

Entityset: clients
  Entities:
    clients [Rows: 25, Columns: 6]
    loans [Rows: 443, Columns: 8]
    payments [Rows: 3456, Columns: 5]
  Relationships:
    No relationships

In [12]:
es['clients']

Entity: clients
  Variables:
    client_id (dtype: index)
    joined (dtype: datetime_time_index)
    income (dtype: numeric)
    credit_score (dtype: numeric)
    join_month (dtype: numeric)
    log_income (dtype: numeric)
  Shape:
    (Rows: 25, Columns: 6)

In [13]:
es['loans']

Entity: loans
  Variables:
    loan_id (dtype: index)
    client_id (dtype: numeric)
    loan_type (dtype: categorical)
    loan_amount (dtype: numeric)
    loan_start (dtype: datetime_time_index)
    loan_end (dtype: datetime)
    rate (dtype: numeric)
    repaid (dtype: categorical)
  Shape:
    (Rows: 443, Columns: 8)

In [14]:
es['payments']

Entity: payments
  Variables:
    payment_id (dtype: index)
    loan_id (dtype: numeric)
    payment_amount (dtype: numeric)
    payment_date (dtype: datetime_time_index)
    missed (dtype: categorical)
  Shape:
    (Rows: 3456, Columns: 5)

In [15]:
r_client_previous = ft.Relationship(es['clients']['client_id'],
                                    es['loans']['client_id'])

es = es.add_relationship(r_client_previous)

In [16]:
r_payments = ft.Relationship(es['loans']['loan_id'],
                                      es['payments']['loan_id'])

es = es.add_relationship(r_payments)

es

Entityset: clients
  Entities:
    clients [Rows: 25, Columns: 6]
    loans [Rows: 443, Columns: 8]
    payments [Rows: 3456, Columns: 5]
  Relationships:
    loans.client_id -> clients.client_id
    payments.loan_id -> loans.loan_id

## Feature Primitives

In [17]:
primitives = ft.list_primitives()
pd.options.display.max_colwidth = 100
primitives[primitives['type'] == 'aggregation'].head(10)

,name,type,description
0,first,aggregation,Determines the first value in a list.
1,skew,aggregation,Computes the extent to which a distribution differs from a normal distribution.
2,avg_time_between,aggregation,Computes the average number of seconds between consecutive events.
3,percent_true,aggregation,Determines the percent of `True` values.
4,num_unique,aggregation,"Determines the number of distinct values, ignoring `NaN` values."
5,entropy,aggregation,Calculates the entropy for a categorical variable
6,all,aggregation,Calculates if all values are 'True' in a list.
7,min,aggregation,"Calculates the smallest value, ignoring `NaN` values."
8,last,aggregation,Determines the last value in a list.
9,mean,aggregation,Computes the average for a list of values.


In [18]:
primitives[primitives['type'] == 'transform'].head()

,name,type,description
22,equal_scalar,transform,Determines if values in a list are equal to a given scalar.
23,divide_numeric,transform,Element-wise division of two lists.
24,less_than_equal_to_scalar,transform,Determines if values are less than or equal to a given scalar.
25,or,transform,Element-wise logical OR of two lists.
26,minute,transform,Determines the minutes value of a datetime.


In [19]:
features, feature_names = ft.dfs(entityset = es, target_entity = 'clients', 
                                 agg_primitives = ['mean', 'max', 'percent_true', 'last'],
                                 trans_primitives = ['year', 'month', 'subtract_numeric', 'divide_numeric'])

In [20]:
pd.DataFrame(features['MONTH(joined)'].head())

,MONTH(joined)
client_id,
42320,4
39384,6
26945,11
41472,11
46180,11


In [21]:
pd.DataFrame(features['MEAN(payments.payment_amount)'].head())

,MEAN(payments.payment_amount)
client_id,
42320,1021.483333
39384,1193.630137
26945,1109.473214
41472,1129.076190
46180,1186.550336


In [22]:
features.head()

,income,credit_score,join_month,log_income,MEAN(loans.loan_amount),MEAN(loans.rate),MAX(loans.loan_amount),MAX(loans.rate),LAST(loans.loan_amount),LAST(loans.loan_type),...,MEAN(loans.rate) / MAX(loans.loan_amount),join_month - log_income / MEAN(payments.payment_amount),log_income / credit_score - income,MEAN(payments.payment_amount) / credit_score - log_income,credit_score - income / LAST(payments.payment_amount),income - join_month / LAST(payments.payment_amount),join_month - log_income / credit_score - log_income,income / MEAN(loans.loan_amount),MAX(payments.payment_amount) / income - join_month,MEAN(payments.payment_amount) / LAST(loans.rate)
client_id,,,,,,,,,,,,,,,,,,,,,
42320,229481,563,4,12.343576,7062.066667,2.457333,13887,6.74,8090,home,...,0.000177,-0.008168,-0.000054,1.855028,-211.569316,212.085952,-0.015152,32.494879,0.012067,321.221174
39384,191204,617,6,12.161096,7865.473684,3.538421,14654,9.23,14654,other,...,0.000241,-0.005162,-0.000064,1.973468,-93.196577,93.495355,-0.010186,24.309280,0.014760,528.154928
26945,214516,806,11,12.276140,7125.933333,2.855333,14593,5.65,9249,cash,...,0.000196,-0.001150,-0.000057,1.397808,-133.819662,134.317470,-0.001608,30.103565,0.012904,387.927697
41472,152214,638,11,11.933043,7510.812500,3.981250,13657,9.82,10122,cash,...,0.000292,-0.000826,-0.000079,1.803443,-104.319339,104.750860,-0.001490,20.265983,0.016005,1096.190476
46180,43851,562,11,10.688553,7700.850000,3.502500,14081,9.26,3834,other,...,0.000249,0.000262,-0.000247,2.152232,-62.107604,62.898135,0.000565,5.694306,0.060675,859.819084


In [23]:
#Depth = 1
pd.DataFrame(features['MEAN(loans.loan_amount)'].head(10))

,MEAN(loans.loan_amount)
client_id,
42320,7062.066667
39384,7865.473684
26945,7125.933333
41472,7510.812500
46180,7700.850000
46109,8951.600000
32885,9920.400000
29841,9813.000000
38537,8986.352941


In [24]:
#Depth = 2
pd.DataFrame(features['LAST(loans.MEAN(payments.payment_amount))'].head(10))

,LAST(loans.MEAN(payments.payment_amount))
client_id,
42320,1192.333333
39384,2311.285714
26945,1598.666667
41472,1427.000000
46180,557.125000
46109,1708.875000
32885,1729.000000
29841,1125.500000
38537,1348.833333


## Automated Deep Feature Synthesis

In [25]:
features, feature_names = ft.dfs(entityset=es, target_entity='clients', 
                                 max_depth = 2)

In [26]:
features.iloc[:, 4:].head()

,SUM(loans.loan_amount),SUM(loans.rate),STD(loans.loan_amount),STD(loans.rate),MAX(loans.loan_amount),MAX(loans.rate),SKEW(loans.loan_amount),SKEW(loans.rate),MIN(loans.loan_amount),MIN(loans.rate),...,MIN(payments.loans.rate),MIN(payments.loans.loan_amount),MEAN(payments.loans.rate),MEAN(payments.loans.loan_amount),NUM_UNIQUE(payments.loans.client_id),NUM_UNIQUE(payments.loans.repaid),NUM_UNIQUE(payments.loans.loan_type),MODE(payments.loans.client_id),MODE(payments.loans.repaid),MODE(payments.loans.loan_type)
client_id,,,,,,,,,,,,,,,,,,,,,
42320,105931,36.86,4165.826885,1.984938,13887,6.74,0.185406,0.993713,1070,0.38,...,0.38,1070,2.523667,7028.058333,1,2,4,42320,1,home
39384,149444,67.23,3964.286840,2.629599,14654,9.23,-0.242626,0.992152,1770,0.43,...,0.43,1770,3.418630,7957.130137,1,2,4,39384,1,cash
26945,106889,42.83,4543.621769,1.619717,14593,5.65,0.174492,-0.002227,653,0.13,...,0.13,653,2.947589,6884.401786,1,2,4,26945,0,credit
41472,120173,63.70,4257.668536,3.198366,13657,9.82,-0.075884,0.416789,986,0.01,...,0.01,986,4.146286,7473.628571,1,2,4,41472,1,other
46180,154017,70.05,3835.726436,2.550263,14081,9.26,0.081292,0.945069,1607,0.57,...,0.57,1607,3.882081,7668.899329,1,2,4,46180,0,other
